# Парсер характеристик 2 линеек: одной - успешной, другой - провальной
Цель тетради: собрать информацию по 2-м линейкам смартфонов:
1. Redmi - популярная в России и за рубежом линейка смартфонов от крупного китайского производителя Xiaomi. Начал рост популярности с 2016 года. Сейчас имеет значительный успех на рынках различных стран.
2. Nokia - популярный бренд смартфонов в 90-х, 2000-х. С 2016 года популярность Nokia начала значительно сокращаться. В настоящий момент предпринимались попытки реанимировать бренд, но они не увенчались успехом.

В резульате должны получиться 2 дадафрейма:
* Redmi - Датафрейм с характеристиками смартфоном Xiaomi Redmi
* Nokia - Датафрейм с характеристиками смартфонов бренда Nokia

Данные надо откуда-то взять.

За парсинг датасета из любого источника на ваш вкус вы получаете 20 баллов
За сбор датасета из разных источников (несколько готовых датасетов, но собранных из разных источников и объединенные общей логикой) вы получаете 10 баллов
За просто готовый датасет – 5 баллов
Если вы взяли готовый датасет и обогатили его, спарсив что-то, вы получаете 20 баллов. Тут важно спарсить что-то полезное, обогащающее датасет. За парсинг book to scrape ради галочки, вы баллов не получите. Не забудьте вставить ссылки на всё, что используете в итоговый блокнот и подробно описать свои действия.

Рекомендация к парсингу: Данное условие не является строгим и обязательным, а скорее рекомендация для вас. Если вы решили парсить данные, рекомендуем убедиться, что вы сможете собрать хотя бы 4-5 числовых признаков и несколько категориальных для вашей модели (если вы собираетесь что-то предсказывать). При этом также рекомендуем собрать хотя бы 1000 наблюдений для модели.

Импортируем библиотеки.

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from random import randint
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore', message='The frame.append method is deprecated and will be removed from pandas in a future version.')

___

# Redmi

## Парсим первый датафрейм - Redmi Sales

In [10]:
# настраиваем драйвер
options = Options()
options.headless = True  # hide GUI
options.add_argument("--window-size=1920,1080")  # set window size to native GUI size
options.add_argument("start-maximized")  # ensure window is full-screen

# Настраиваем браузер, чтобы он не грузил Java и картинки
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options, chrome_options=chrome_options)

# Делаем запрос
driver.get("https://www.ebay.com/sh/research?marketplace=EBAY-US&keywords=Redmi&dayRange=730&endDate=1684047600000&startDate=1620975600000&categoryId=9355&offset=0&limit=50&tabName=SOLD&tz=Europe%2FMoscow")

#Ждём загрузку
time.sleep(5)

C:\Users\druny\AppData\Local\Temp\ipykernel_24992\1312355097.py:3: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True  # hide GUI
C:\Users\druny\AppData\Local\Temp\ipykernel_24992\1312355097.py:9: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(options=options, chrome_options=chrome_options)


Тут мы ручками авторизируемся и заходим

Парсим названия и числовые характеристики (Цены, продажи)

In [17]:
def table_parser(soup: BeautifulSoup):
    page_data = pd.DataFrame()
    for product in soup.find_all('tr', class_='research-table-row'):
        title = product.find('span', attrs={'data-item-id': True}).text
        price = product.find('td', class_='research-table-row__item research-table-row__avgSoldPrice').div.find_all('div')[0].text.strip()
        shipping = product.find('td', class_='research-table-row__item research-table-row__avgShippingCost').div.find_all('div')[0].text.strip()
        sales_count = product.find('td', class_='research-table-row__item research-table-row__totalSoldCount').div.find_all('div')[0].text.strip()
        sales_value = product.find('td', class_='research-table-row__item research-table-row__totalSalesValue').div.find_all('div')[0].text.strip()
        last_sale = product.find('td', class_='research-table-row__item research-table-row__dateLastSold').div.find_all('div')[0].text.strip()
        try:
            link = product.find('a', class_='research-table-row__link-row-anchor', attrs={'href': True})['href']
        except:
            link = None

        data = {"Title": title, "Price": price, "Shipping_cost": shipping, "Sales_Count": sales_count, "Sales_Value": sales_value, "Link": link, "Last_Sale": last_sale}
        page_data = page_data.append(data, ignore_index=True)
    return page_data

In [18]:
df = pd.DataFrame()
for _ in tqdm(range(100)): #Решено спарсить 100 страниц
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    data = table_parser(soup)
    df = pd.concat([df, data], ignore_index=True)
    next_button = driver.find_element(By.XPATH, '//button[@class="pagination__next"]')
    next_button.click()
    time.sleep(randint(5, 15))
    pass

100%|██████████| 100/100 [19:36<00:00, 11.76s/it]


In [20]:
df.head()

,Title,Price,Shipping_cost,Sales_Count,Sales_Value,Link,Last_Sale
0,Wall Outlet To Triple USB Power Adapter - Wall...,$9.99,$12.91,940,"$9,388.60",https://www.ebay.com/itm/363346871915?nordt=tr...,"May 11, 2023"
1,Xiaomi Redmi Note 11 128GB 4GB RAM GSM Unlocke...,$187.39,$7.10,772,"$144,668.87",https://www.ebay.com/itm/325013923406?nordt=tr...,"May 14, 2023"
2,Xiaomi Redmi Note 10 Pro 128GB 6GB RAM GSM FAC...,$282.12,$12.05,455,"$128,364.84",https://www.ebay.com/itm/324648557855?nordt=tr...,"May 5, 2023"
3,Xiaomi Redmi Note 11 Pro+ 5G Plus 256GB 8GB RA...,$395.19,$12.25,408,"$161,235.53",https://www.ebay.com/itm/203898287868?nordt=tr...,"May 10, 2023"
4,Xiaomi Redmi Note 10 *5G* 128GB 4GB RAM GSM Un...,$195.45,$8.60,428,"$83,654.10",https://www.ebay.com/itm/324618206904?nordt=tr...,"Jan 18, 2023"


In [21]:
df.to_csv("Redmi Sales.tsv", sep='\t', index=False)

___

## Парсим характеристики моделей

In [2]:
df=pd.read_csv("Redmi Sales.tsv", delimiter = "\t")

In [3]:
Links = df.Link.tolist()
Links[0:10]

['https://www.ebay.com/itm/363346871915?nordt=true&orig_cvip=true&rt=nc',
 'https://www.ebay.com/itm/325013923406?nordt=true&orig_cvip=true&rt=nc',
 'https://www.ebay.com/itm/324648557855?nordt=true&orig_cvip=true&rt=nc',
 'https://www.ebay.com/itm/203898287868?nordt=true&orig_cvip=true&rt=nc',
 'https://www.ebay.com/itm/324618206904?nordt=true&orig_cvip=true&rt=nc',
 'https://www.ebay.com/itm/194060416913?nordt=true&orig_cvip=true&rt=nc',
 'https://www.ebay.com/itm/304041013742?nordt=true&orig_cvip=true&rt=nc',
 'https://www.ebay.com/itm/324254026178?nordt=true&orig_cvip=true&rt=nc',
 'https://www.ebay.com/itm/324610842527?nordt=true&orig_cvip=true&rt=nc',
 'https://www.ebay.com/itm/144361594762?nordt=true&orig_cvip=true&rt=nc']

In [4]:
while np.nan in Links:
    Links.remove(np.nan)
None in Links

False

In [5]:
len(Links)

905

In [7]:
import time
import random

characteristics_df = pd.DataFrame()
undone = []

# Задержка перед каждым запросом
def delay():
    time.sleep(random.uniform(1, 5))  # Измените значения задержки по своему усмотрению

def req_parser(url):
    global characteristics_df, undone

    try:
        # Отправляем GET-запрос и получаем ответ
        response = requests.get(url)

        # Получаем HTML-код страницы
        html = response.text

        # Создаем объект BeautifulSoup
        soup = BeautifulSoup(html, 'html.parser')

        # Находим родительский элемент
        parent_element = soup.find('div', class_='ux-layout-section--features')

        # Initialize an empty dictionary to store the characteristics
        characteristics = {"Link": url}
        done = []

        # Find all the characteristic rows
        rows = parent_element.find_all('div', class_='ux-layout-section-evo__row')

        # Iterate over each row and extract the characteristic name and value
        for row in rows:
            name_element = row.find_all('div', class_='ux-labels-values__labels')
            value_element = row.find_all('div', class_='ux-labels-values__values-content')

            for name, value in zip(name_element, value_element):
                if name and value:
                    name_text = name.get_text(strip=True)
                    value_text = value.get_text(strip=True)
                    if name_text not in done and value_text not in done:
                        characteristics[name_text] = value_text
                        done.append(name_text)
                        done.append(value_text)

        characteristics_df = characteristics_df.append(characteristics, ignore_index=True)

        # Задержка перед следующим запросом
        delay()

    except:
        print("Нащальника, не получилось:", url)
        undone.append(url)


for url in tqdm(Links):
    req_parser(url)

characteristics_df.to_csv("Redmi Characteristics.tsv", sep='\t', index=False)

 13%|█▎        | 121/905 [10:09<1:20:13,  6.14s/it]

Нащальника, не получилось: https://www.ebay.com/itm/374563147366?nordt=true&orig_cvip=true&rt=nc


 15%|█▍        | 132/905 [10:56<44:11,  3.43s/it]  

Нащальника, не получилось: https://www.ebay.com/itm/314504650326?nordt=true&orig_cvip=true&rt=nc


 19%|█▉        | 175/905 [14:40<1:06:20,  5.45s/it]

...

 89%|████████▉ | 806/905 [1:02:50<05:44,  3.48s/it]

Нащальника, не получилось: https://www.ebay.com/itm/234940966290?nordt=true&orig_cvip=true&rt=nc


 89%|████████▉ | 807/905 [1:02:52<04:52,  2.99s/it]

Нащальника, не получилось: https://www.ebay.com/itm/234941635444?nordt=true&orig_cvip=true&rt=nc


100%|██████████| 905/905 [1:10:55<00:00,  4.70s/it]
